In [1]:
# Import the numpy library for numerical operations
import numpy as np

# Import the OpenCV library for computer vision tasks
import cv2

# Import the os library to interact with the operating system
import os

# Import TensorFlow, a machine learning and neural network library
import tensorflow as tf

# Import Keras, a high-level neural networks API running on top of TensorFlow
import keras

# From the keras.layers module, import various layers used in neural network architectures
from keras.layers import Conv2D, Flatten, BatchNormalization, Dense, MaxPooling2D, Dropout

# Import train_test_split function to divide data into training and testing sets
from sklearn.model_selection import train_test_split

# Import to_categorical for converting a class vector to binary class matrix
from keras.utils import to_categorical

# Import ImageDataGenerator from TensorFlow Keras for real-time data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import LabelEncoder for encoding labels with a value between 0 and n_classes-1
from sklearn.preprocessing import LabelEncoder

# Import Adam optimizer, a method for stochastic optimization
from keras.optimizers import Adam

# Import Sequential from Keras, a linear stack of neural network layers
from keras.models import Sequential

# Import regularizers from TensorFlow Keras, providing penalties on layer parameters during optimization
from tensorflow.keras import regularizers

# Import various performance metrics from sklearn.metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# Import confusion_matrix to evaluate the accuracy of a classification
from sklearn.metrics import confusion_matrix

# Import seaborn, a statistical data visualization library based on matplotlib
import seaborn as sns

# Import matplotlib.pyplot for creating static, animated, and interactive visualizations in Python
import matplotlib.pyplot as plt

from matplotlib import cm

In [2]:
# Loading dataset
data_dir = "C:/Users/ipman/OneDrive/Desktop/Data Mining/assignment2/archive/chest_xray/train" # Hardcoded path on my local machine, relative path sometimes outputs error

In [3]:
# Accessing classes in data directory
sub_folders = os.listdir(data_dir)
print(len(sub_folders))

2


NORMAL and PNEUMONIA subfolders in "train" folder

In [4]:
# Create an empty list named 'images' to store image data
images = []

# Create an empty list named 'labels' to store the labels corresponding to the images
labels = []

In [5]:
for sub_folder in sub_folders:
    label = sub_folder
    
    # Constructing the path to the current sub_folder
    path = os.path.join(data_dir, sub_folder)
    
    # Listing all the images in the sub_folder
    sub_folder_images = os.listdir(path)
    
    for image_name in sub_folder_images:
        
        # Constructing the path of current image
        image_path = os.path.join(path, image_name)
        
        # Loading the image using OpenCV
        img = cv2.imread(image_path)
        
        # Resizing image
        img = cv2.resize(img, (256, 256))
        
        # Append the images to the image list 
        images.append(img)
        
        # Append the labels
        labels.append(label)

In [6]:
# list of images and labels to the numpy array
images = np.array(images)
labels = np.array(labels)

In [7]:
# Split the dataset 
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 42)

In [8]:
# Image Preprocessing
def preprocessing(img):
    img = img / 255.0
    return img

In [9]:
# Applying the preprocessing to the entire dataset
X_train = np.array(list(map(preprocessing, X_train)))
X_val = np.array(list(map(preprocessing, X_val)))
X_test = np.array(list(map(preprocessing, X_test)))

In [14]:
# Import the ImageDataGenerator class from Keras for data preprocessing
from keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
data_gen = ImageDataGenerator(
    width_shift_range=0.1,  # Maximum shift in the width of the image, as a fraction of total width (10% here)
    height_shift_range=0.1, # Maximum shift in the height of the image, as a fraction of total height (10% here)
    rotation_range=10,      # Maximum degree of random rotations of the image
    shear_range=0.1         # Shear Intensity (Shear angle in counter-clockwise direction as radians)
)

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (C:\Users\ipman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\preprocessing\image\__init__.py)

In [ ]:
# Label Encoder
label_encoder = LabelEncoder()
label_encoder.fit(labels)

In [ ]:
# Encode the labels
y_train = label_encoder.transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

In [ ]:
# Number of Classes
num_classes = len(label_encoder.classes_)

In [ ]:
# Converting the labels into One-Hot encoding
y_train = to_categorical(y_train, num_classes = num_classes)
y_val = to_categorical(y_val, num_classes = num_classes)
y_test = to_categorical(y_test, num_classes = num_classes)

In [ ]:
# Function to build the CNN model
def build_model():
    model = Sequential()  # Initialize a sequential model

    # First convolutional layer with 32 filters of size 5x5, using same padding, stride of 1, and ReLU activation
    model.add(Conv2D(32, (5,5), strides=(1,1), padding='same', activation='relu', input_shape=(256, 256, 3)))
    # First max pooling layer with pool size of 2x2
    model.add(MaxPooling2D(2,2))
    # Add dropout layer to prevent overfitting, dropping out 50% of the units
    model.add(Dropout(0.5))
    
    # Second convolutional layer with 64 filters of size 3x3, same padding, stride of 1, and ReLU activation
    model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.5))
    
    # Third convolutional layer with 128 filters, same settings
    model.add(Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.5))
    
    # Fourth convolutional layer with 256 filters
    model.add(Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.5))
    
    # Fifth convolutional layer with 512 filters
    model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.5))
    
    # Sixth convolutional layer with 32 filters (reducing complexity perhaps for specific feature extraction)
    model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.5))
    
    # Flatten the output of the last pooling layer to feed into the dense layer
    model.add(Flatten())
    
    # Dense layer with 512 neurons and ReLU activation
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    # Another dense layer with 1024 neurons
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer with a number of neurons equal to the number of classes, using sigmoid activation
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Optimizer configuration - using Adam optimizer with a learning rate of 0.0001
    optimizer = Adam(learning_rate=0.0001)
    # Compile the model with binary crossentropy loss and accuracy as the metric
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model  

In [ ]:
model = build_model() # Call the build_model function to create and configure the CNN model

print(model.summary())

The model has a total of 2,507,618 parameters, all of which are trainable

In [ ]:
# Debbuging / Checking compatibility for tensorflow GPU computing

import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("All devices:", tf.config.list_physical_devices())
else:
    print("No GPU is detected.")

I have tried CPU and GPU computing to see if there is any difference in results, there is none.

<b> NOTE THAT MY CPU IS OVERCLOCKED, its a beast :) </b>

import json

# Train the model
history = model.fit(data_gen.flow(X_train, y_train, batch_size = 32),
                   validation_data = (X_val, y_val),
                   epochs = 30,
                   verbose = 2)

model.save('my_model.keras') # Saving the model in order to reuse it without training the model again

history_dict = history.history # Saving the history in order to reuse it without training the model again

with open('history.json', 'w') as f:
    json.dump(history_dict, f)

<b>RUN THIS CELL AGAIN IF YOU WANT TO TRAIN THE MODEL AGAIN, OTHERWISE, I HAVE SAVED THE MODEL AS "my_model" AND HISTORY AS "history".</b>

In [ ]:
from keras.models import load_model # Loading saved model


model = load_model('my_model.keras')

In [ ]:
import json # Loading saved history 

with open('history.json', 'r') as f:
    history = json.load(f)

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_accuracy(history):
    # Check if 'history' is a dict or has a .history attribute
    if isinstance(history, dict):
        hist = history  # If already a dict, use it directly
    else:
        hist = history.history  # If a history object, use the .history attribute

    # Plot training & validation loss values
    plt.plot(hist['loss'])
    plt.plot(hist['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    for i, loss in enumerate(hist['val_loss']):
        plt.text(i, loss, f'{loss:.2f}', ha='center', va='bottom', fontsize=7)

    plt.show()

    # Plot training & validation accuracy values
    plt.plot(hist['accuracy'])
    plt.plot(hist['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    for i, acc in enumerate(hist['val_accuracy']):
        plt.text(i, acc, f'{acc:.2f}', ha='center', va='bottom', fontsize=7)

    plt.show()

plot_loss_accuracy(history)

# plot_loss_accuracy(model_history)  # Use this if history is from model.fit()

Model Loss:

This graph displays the loss metric of the model on both the training set and the validation set.

Both losses decrease over time, which is a good indicator that the model is learning.

Initially, the training loss decreases more sharply than the validation loss, indicating the model is quickly learning from the training data.

After the sharp decrease, the validation loss flattens and remains relatively constant, suggesting that the model might not be improving much on the validation set beyond this point.

The training loss continues to slightly decrease or remains flat, which could be a sign that the model has nearly converged on the training data.

Throughout the process, the validation loss remains higher than the training loss, which is typical in most training scenarios due to the model being optimized on the training data.

Model Accuracy:

The second graph shows the accuracy metric for both the training set and validation set.

The accuracy on both datasets increases sharply at the beginning, which indicates a good initial learning phase.

The training accuracy continues to increase and plateaus near a high value, which indicates a good fit on the training data.

The validation accuracy increases along with the training accuracy and then fluctuates slightly around a certain value but generally remains flat after about 15 epochs, suggesting that the model is not significantly improving after this point.

There is a noticeable gap between training and validation accuracy, suggesting some overfitting. However, the gap does not widen as the epochs increase, which is a good sign.

Overall:

The model shows effective learning patterns with both metrics improving.

The plateau in validation metrics suggests potential limits reached by the current model configuration.

Slight overfitting is observed, but it doesn't worsen over time, indicating stable generalization.

In [ ]:
# Predicting labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Converting one-hot encoded test labels back to categorical labels
y_true = np.argmax(y_test, axis=1)

# Generating the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Visualizing the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

True Positives (TP): The lower right cell shows 710 instances where the model correctly predicted 'Pneumonia'. This indicates a high number of correct predictions for this class.

True Negatives (TN): The upper left cell shows 281 instances where the model correctly identified 'Normal'. This suggests that the model is reasonably good at identifying the negative class as well.

False Positives (FP): The upper right cell has a value of 6, indicating that there were only 6 instances where the model incorrectly predicted 'Pneumonia' when the actual label was 'Normal'. This is quite low, showing that the model doesn't often mistake healthy cases for Pneumonia.

False Negatives (FN): The lower left cell shows 47 instances where the model predicted 'Normal' but the true label was 'Pneumonia'. These are critical errors in a medical context, as they represent missed diagnoses.

In summary, the confusion matrix suggests the model has a strong predictive ability, with a high number of true positives and true negatives and relatively low false positives and false negatives. 

In [ ]:
# Accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Precision
precision = precision_score(y_true, y_pred_classes)

# Recall
recall = recall_score(y_true, y_pred_classes)

# F1 Score
f1 = f1_score(y_true, y_pred_classes)

# ROC Curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_pred_classes)
roc_auc = auc(fpr, tpr)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"AUC: {roc_auc:.2f}")

# Visualization of ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

ROC Curve: The solid orange line represents the ROC curve of the model, with the area under the curve (AUC) being 0.96. <b>This is a very good score as the AUC ranges from 0 to 1, with 1 representing a perfect model.</b>

True Positive Rate (TPR): Also known as recall, it's on the y-axis and indicates the proportion of actual positives that are correctly identified as such.

False Positive Rate (FPR): Plotted on the x-axis, it represents the proportion of actual negatives that are incorrectly identified as positives.

Diagonal Dashed Line: The blue dashed line represents the ROC curve of a purely random classifier; a good classifier stays as far away from this line as possible.

Performance Metrics:

Accuracy: The model correctly predicts both pneumonia and normal cases 95% of the time.

Precision: Out of all cases predicted as pneumonia, 99% are truly pneumonia cases.

Recall: The model correctly identifies 94% of all actual pneumonia cases.

F1 Score: With an F1 score of 0.96, the model has a high balance between precision and recall.

<b>AUC: An AUC of 0.96 implies a high chance that the model will be able to distinguish between pneumonia and normal cases.</b>

<b>Using LIME for Local Interpretability</b>

In [ ]:
from lime import lime_image  # Module for explaining predictions on image data
from skimage.segmentation import mark_boundaries  # Function to outline segments in an image

# Function to explain an instance with LIME for image data
def explain_instance_with_lime(image, model, num_features=10, hide_rest=False):
   
    # Create an explainer for images
    explainer = lime_image.LimeImageExplainer()
    
    # Generate explanation for the specific image
    explanation = explainer.explain_instance(image.astype('double'), model.predict,
                                             top_labels=5, hide_color=0, num_samples=1000)

    # Get image and mask for the top prediction
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0], positive_only=False, num_features=num_features, hide_rest=hide_rest
    )

    # Visualize the image and the mask
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.title(f'LIME Explanation')
    plt.axis('off')
    
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='red', edgecolor='r', label='Positive Regions'),
                       Patch(facecolor='green', edgecolor='g', label='Negative Regions')]
    plt.legend(handles=legend_elements, loc='lower right')

    plt.show()

   
explain_instance_with_lime(X_test[50], model)  # Change the vale in X_test[] to analyze a different image

The function 'explain_instance_with_lime', utilizes the LIME framework to interpret and explain the predictions of an image-based machine learning model. The explanation aims to make the model's decision-making process transparent by identifying which parts of the input image influence the model's predictions.

Red areas indicate features (or superpixels) that the model found to be positively correlated with the predicted class. 

Green areas  represent features that negatively influence the prediction. 

<b>Using Grad-CAM for Visualizing Important Regions</b>

In [ ]:
from tensorflow.keras.models import Model

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a model that outputs the last convolutional layer and the final model predictions
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    # Using tf.GradientTape to track operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Run the model and get the outputs of the last convolutional layer and the final predictions
        last_conv_layer_output, preds = grad_model(img_array)
        
        # If no specific prediction index is provided, use the class with highest score
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        
        # Get the output of the model for the class index that we're interested in
        class_channel = preds[:, pred_index]

    # Compute gradients of the target class output 
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Global average pooling of the gradients across the width and height dimensions
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight the output feature map with the computed pooled gradients
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)  # Remove dimensions of size 1

    # Apply ReLU to the heatmap and normalize it
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()  # Convert tensor to numpy array for visualization

heatmap = make_gradcam_heatmap(np.expand_dims(X_test[50], axis=0), model, 'conv2d_5')

# Display the heatmap
plt.matshow(heatmap)
plt.show()

Model Creation: The function starts by defining a new Model that takes the same inputs as the original model but outputs both the activations from the last convolutional layer (last_conv_layer_name) and the final predictions. This allows the function to access intermediate activations needed for Grad-CAM.

Gradient Computation: Using tf.GradientTape, which records operations for automatic differentiation, the function computes the gradient of the class output (which we're interested in) with respect to the activations of the last convolutional layer.

Pooling Gradients: It then pools these gradients along the spatial dimensions (width and height), which gives a small set of numbers (one per channel) that represent the importance of each channel regarding the target class.

Creating the Heatmap: These pooled gradients are then used to weigh the convolutional layer's output channels, and a spatial average is computed to obtain the heatmap. 

<b>The heatmap shows the regions of the input image that were most important for predicting the target class.</b>

Post-processing: The heatmap is then processed through a ReLU function and normalized to make it visually interpretable.

<b>Visualizing the Grad-CAM Output</b>

In [ ]:
def display_gradcam(img, heatmap, alpha=0.4):
    # Ensure the image is scaled to [0, 255] and converted to uint8
    img = np.uint8(255 * img)

    # Convert to RGB if necessary (assuming the original images are in BGR format)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Rescale heatmap to a range 0-255 and convert to uint8
    heatmap = np.uint8(255 * heatmap)

    # Get the Jet colormap
    jet = cm.get_cmap("jet")

    # Get RGB values from the colormap for each point in the heatmap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Resize the heatmap to match the image size
    jet_heatmap = cv2.resize(jet_heatmap, (img.shape[1], img.shape[0]))
    jet_heatmap = np.uint8(255 * jet_heatmap)  # Ensure jet_heatmap is uint8

    # Combine the heatmap with the original image
    superimposed_img = jet_heatmap * alpha + img

    # Display the superimposed image
    plt.imshow(superimposed_img / 255)
    plt.axis('off')
    plt.show()

# Call the display function with the test image and heatmap
display_gradcam(X_test[50], heatmap)

Image Scaling and Conversion: The image is first scaled to the range [0, 255] and converted to 8-bit integers, as this is the expected format for images in many visualization and image processing functions.

Color Space Conversion: Converts the image from BGR (default color space for images loaded by OpenCV) to RGB to match the color order used by Matplotlib.

Heatmap Scaling: The heatmap, which is originally in a floating-point format where values range from 0 to 1, is scaled to [0, 255] to match the color depth of standard images.

Colormap Application: The Jet colormap is applied to the heatmap. This step converts the single-channel heatmap values into a three-channel (RGB) format, which can be visualized in color.

Heatmap Resizing: The heatmap is resized to match the dimensions of the original image. This is necessary to ensure that the heatmap accurately overlays the corresponding parts of the original image.

Superimposing Heatmap: The colored heatmap is then superimposed on the original image. The alpha parameter controls the transparency of the heatmap overlay, allowing you to adjust how much of the original image is visible underneath the heatmap.

Display: The superimposed image is displayed using Matplotlib. The division by 255 is to convert the image back into the [0, 1] range expected by Matplotlib.

<b> Areas of Focus: The brightly colored areas indicate regions that the neural network focused on when making its decision. These areas are presumably where the model detected patterns or features strongly associated with the presence of pneumonia.</b>

<b>Color Gradient: The colors range from blue to red, where blue represent areas of less importance and red signifies areas of high importance. The yellow and red areas are where the model is 'looking' to make its decision about the presence of pneumonia in the chest X-ray.</b>

<b>Assessment of Model Behavior: The fact that the heatmap is concentrated over the lung fields is a good sign, it suggests that the model is paying attention to the relevant parts of the X-ray for diagnosing pneumonia. </b>